# Text Generation

### José Pablo Kiesling Lange - 21581

In [1]:
from datasets import load_dataset

c:\Users\TheKi\OneDrive - UVG\Semestre X\Procesamiento de Lenguaje Natural\text-generation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

In [3]:
dataset_train = dataset["train"]["text"]
dataset_test = dataset["test"]["text"]

## Text Normalization

In [4]:
def normalize_text(text):
    text = ''.join(char for char in text if char.isalpha() or char.isspace() or char in ".,!?;:'\"-")
    text = ' '.join(word for word in text.split() if all(ord(char) < 128 for char in word))
    text = ' '.join(text.split())
    return text 

In [5]:
dataset_train = [normalize_text(text) for text in dataset_train if text.strip() != '']
dataset_test = [normalize_text(text) for text in dataset_test if text.strip() != '']

## Feed Forward Neural Network